# PLN I

## Desafio 1

Se utilizará de base para realizar el desafío la notebook planteada en clase.

Importamos las bilbiotecas que se utilizarán en todo el desafío

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import cross_val_score
import optuna

from sklearn.datasets import fetch_20newsgroups
import numpy as np

c:\Users\gaston.e_parsimotion\anaconda3\envs\EIA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Punto 1: similitud entre documentos

Cargo y dividos los datos en train y test

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

Instanciamos un vectorizador. Utilizo los parámetros:
- "stop_words='english'": para no considerar palabras comunes y poco informativas

In [3]:
tfidfvect = TfidfVectorizer(stop_words='english')

Luego, genero el set de entrenamiento a partir del vectorizador

In [4]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

A continuación, obtengo 5 documentos arbitrareamente y, para cada uno de ellos:
- Mido su similitud coseno contra todo los elementos del set
- Me quedo con los 5 más cercanos
- Los muestro para poder evaluarlos

In [5]:
documents_to_analyze_idx = [33,551,3123,11045,5312]

Defino function para poder calcular la similitud coseno y obtener los 5 documentos más cercanos. Para cada uno de ellos, muestro la comparación contra clase a la cual pertenecen e imprimo su contenido para evaluar.

In [6]:
def show_nearest_five(baseDocumentIdx):
    cossim = cosine_similarity(X_train[baseDocumentIdx], X_train)[0]
    mostsim = np.argsort(cossim)[::-1][1:6] # no considero al propio documento, por eso arranco de 1
    
    print("---------------------------\n")
    print("Clase de documento base      :",newsgroups_train.target_names[y_train[baseDocumentIdx]])
    for i,similarDocumentIdx in enumerate(mostsim):
        print(f"Clase de documento cercano {i} :",newsgroups_train.target_names[y_train[similarDocumentIdx]])

    print("----------------------------------")
    print("\nDocBase: \n",newsgroups_train.data[baseDocumentIdx])

    for i,similarDocumentIdx in enumerate(mostsim):
        print("----------------------------------")
        print(f"\nDoc cercano {i} :\n",newsgroups_train.data[similarDocumentIdx])

    

Luego, vemos los resultados para los 5 documentos definidos

In [7]:
for i in documents_to_analyze_idx:
 show_nearest_five(i)

---------------------------

Clase de documento base      : talk.politics.mideast
Clase de documento cercano 0 : talk.politics.mideast
Clase de documento cercano 1 : talk.politics.mideast
Clase de documento cercano 2 : talk.politics.mideast
Clase de documento cercano 3 : sci.electronics
Clase de documento cercano 4 : talk.politics.mideast
----------------------------------

DocBase: 
 
How do you define war?  Do seiges and constant attacks on villiages
count as acts of war, or is that only when the Jews do them?
January, 1948: Arab Liberation Army attacks Kfar Szold
               1000 men attack Kfar Etzion, 14 miles south of Jerusalem,
                    after cutting off the supply lines to it.
Attacks on Yehiam (Western Galilee) and kibbutz Tirat Tzvi.
By Mid-March, The Jewish settlements in the Negev had been cut off from
      land links with the rest of the Jewish population.
         The Etzion group of villiages, near Hebron, had been cut off,
            while 42 members of 

**Conclusiones:**

A primera vista se observa un buen resultado al calcular la similitud coseno entre documentos. A pesar de que se obtuvo un resultado imperfecto, verifica un porcentaje de aciertos no menor para el número de clases que existen. Adicionalmente, para muchos de los casos en los que no hay aciertos, las clases de los documentos detectados como "similares" tienen contenidos que comparten cierta cercanía con el documento base, a pesar de no ser de la misma clase exacta.

### Punto 2: Naive - Bayes

Instanciamos el modelo de Naive Bayes multinomial y entrenamos con el dataset generado previamente

In [8]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

Luego, instancio un nuevo vectorizador, sobre el cual realizaré pruebas para modificar los hiperparámetros de max_df y min_df, de forma tal que pueda obtener la mejor resultado para la métrica F1. 

Para realizar las distintas comprobaciones de hiperparámetros del vectorizador utilizo optuna. 

In [9]:
f1_scorer = make_scorer(f1_score, average='macro')

def eval_hiperparameters_with_model(model):
 def objective(trial):
     max_df = trial.suggest_float("max_df", 0.5, 1.0)
     min_df = trial.suggest_float("min_df", 1e-5, 0.1)
 
     vectorizer = TfidfVectorizer(stop_words='english', max_df=max_df, min_df=min_df)
     X_train_nb = vectorizer.fit_transform(newsgroups_train.data)
 
     clf = model
 
     score = cross_val_score(clf, X_train_nb, y_train, scoring=f1_scorer, cv=3).mean()
     return score
 
 study = optuna.create_study(direction="maximize")
 study.optimize(objective, n_trials=30)
 
 print("\nMejores hiperparámetros:")
 print(study.best_params) 
 print(f"Mejor F1-score: {study.best_value:.4f}")

In [10]:
ml_nb = MultinomialNB()
eval_hiperparameters_with_model(ml_nb)

[I 2025-05-10 19:39:27,036] A new study created in memory with name: no-name-31f22069-ed7c-4e9d-a0e9-894ba292aa88
[I 2025-05-10 19:39:29,532] Trial 0 finished with value: 0.15104355611634704 and parameters: {'max_df': 0.5396089872159779, 'min_df': 0.055779971966329556}. Best is trial 0 with value: 0.15104355611634704.
[I 2025-05-10 19:39:31,883] Trial 1 finished with value: 0.37333879828394667 and parameters: {'max_df': 0.9576326303145575, 'min_df': 0.02327036768987728}. Best is trial 1 with value: 0.37333879828394667.
[I 2025-05-10 19:39:34,235] Trial 2 finished with value: 0.5107361714461944 and parameters: {'max_df': 0.7374335138170599, 'min_df': 0.012767624516513254}. Best is trial 2 with value: 0.5107361714461944.
[I 2025-05-10 19:39:36,735] Trial 3 finished with value: 0.10062409574193387 and parameters: {'max_df': 0.749641262904885, 'min_df': 0.09377547782623508}. Best is trial 2 with value: 0.5107361714461944.
[I 2025-05-10 19:39:39,190] Trial 4 finished with value: 0.629554700


Mejores hiperparámetros:
{'max_df': 0.5406584391067495, 'min_df': 0.00019355288378657218}
Mejor F1-score: 0.6908


A partir de los resultados obtenidos, planteamos que la configuración de parámetros para el vectorizador que optimiza la f1_score para el modelo multinomial son:

- max_df = 0.9364523716744748
- min_df = 0.00025908097580620935

Luego, verifico para el modelo complementBN

In [11]:
c_nb = ComplementNB()
eval_hiperparameters_with_model(c_nb)

[I 2025-05-10 19:40:41,740] A new study created in memory with name: no-name-fbc223aa-f069-4055-b627-f3890b7f0292
[I 2025-05-10 19:40:43,979] Trial 0 finished with value: 0.20354778476957644 and parameters: {'max_df': 0.8829558943779956, 'min_df': 0.04120238766247969}. Best is trial 0 with value: 0.20354778476957644.
[I 2025-05-10 19:40:46,214] Trial 1 finished with value: 0.10360189049555253 and parameters: {'max_df': 0.6656960518794157, 'min_df': 0.07749070369058683}. Best is trial 0 with value: 0.20354778476957644.
[I 2025-05-10 19:40:49,937] Trial 2 finished with value: 0.7304410640273548 and parameters: {'max_df': 0.868375426037917, 'min_df': 0.0005173198243831582}. Best is trial 2 with value: 0.7304410640273548.
[I 2025-05-10 19:40:53,713] Trial 3 finished with value: 0.08855054486930188 and parameters: {'max_df': 0.6348584481361792, 'min_df': 0.09523393410839141}. Best is trial 2 with value: 0.7304410640273548.
[I 2025-05-10 19:40:56,412] Trial 4 finished with value: 0.096734951


Mejores hiperparámetros:
{'max_df': 0.7863459782887479, 'min_df': 0.00011846597610316265}
Mejor F1-score: 0.7512


A partir de los resultados obtenidos, planteamos que la configuración de parámetros para el vectorizador que optimiza la f1_score para el modelo complementNB son:

- max_df = 0.9403240472063371
- min_df = 8.944305353494775e-05

Siendo que el modelo complementNB obtuvo un F1-score de 0.7512 y que el modelo multinomialNB obtuvo un F1-score de 0.6908, de necesitar utilizar el modelo para inferir nuevos datos, entrenaríamos el modelo complementNB utilizando los parámetros obtenidos por optuna.

### Punto 3: similitud entre palabras transponiendo matriz documento-término

Primero, obtenemos la matriz documento-término en función de lo evaluado previamente

In [12]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9403240472063371, min_df=8.944305353494775e-05)
X_train_nb = vectorizer.fit_transform(newsgroups_train.data)

Traspongo la matriz documento-término para obtener la matriz término-documento y poder evaluar la similitud entre palabras a partir de sus vectores "descriptores"

In [13]:
ter_doc_matrix = X_train_nb.T

Creo diccionario para poder encontrar las palabras una vez obtenidos los índices de las 5 más similares.

In [14]:
idx2word = {v: k for k,v in vectorizer.vocabulary_.items()}

In [15]:
words_to_analyze = ["eye","door","white","potato","gear"]

In [16]:
def show_nearest_five_words(baseWord):
    idx = vectorizer.vocabulary_[baseWord]
    base_vector = ter_doc_matrix[idx:idx+1]

    cossim_ter = cosine_similarity(base_vector, ter_doc_matrix)[0]
    mostsim_ter = np.argsort(cossim_ter)[::-1][1:6]
    
    print("base word   : ",baseWord)
    for similar_word_idx in mostsim_ter:
        print("Similar word: ",idx2word[similar_word_idx])

Luego, observamos las palabras similares para cada una de las 5 palabras elegidas

In [17]:
for word in words_to_analyze:
    show_nearest_five_words(word)
    print("\n")

base word   :  eye
Similar word:  dominant
Similar word:  stare
Similar word:  dominance
Similar word:  percieve
Similar word:  handedness


base word   :  door
Similar word:  openers
Similar word:  secluded
Similar word:  genie
Similar word:  prelude
Similar word:  scratchy


base word   :  white
Similar word:  house
Similar word:  sox
Similar word:  black
Similar word:  rednecks
Similar word:  nigger


base word   :  potato
Similar word:  oversight
Similar word:  dep
Similar word:  succeeding
Similar word:  evolving
Similar word:  crippled


base word   :  gear
Similar word:  minivan
Similar word:  legacy
Similar word:  climb
Similar word:  throttle
Similar word:  viper




**Conclusiones:**

A partir del análisis realizado con la matriz traspuesta y la posterior evaluación de la similitud coseno entre los vectores que esperábamos tuvieran algun grado de representatividad para las palabras, podemos mencionar que hay elementos para plantear que existe información encapsulada en dichos vectores, ya que muchas de las palabras con mayor grado de similitud tienen una conexión semántica con la palabra base. Dado el amplio vocabulario existente en este corpus, esto es un indicativo de que los vectores obtenido de la transposición realizada contienen información relevante respecto a los términos que representan.

Sin embargo, a pesar de observar un cierto grado de representación, es importante notar que muchos de los resultados obtenidos no poseen una conexión estrecha con la palabra base, por lo que no se considera que este método, al menos en los términos planteados, pueda utilizarse de forma general para plantear similitud entre palabras, sino que se necesitaría buscar algún mecanismo de mejora o postular algún otro método que logre capturar de manera más precisa lo buscado.